In [1]:
%run ./packer.ipynb

In [2]:
import pandas as pd
import numpy as np
import csv
import time
import random
import matplotlib.pyplot as plt
from gurobipy import *  
import math
import scipy.interpolate as interp
from mpl_toolkits import mplot3d
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from matplotlib import animation

In [3]:
data = pd.read_excel('Data.xlsx') #All Cardboard Information
#If individual cardboard are to be analyzed 
#data = pd.read_excel('Data_New.xlsx') #Create a file with the information of only that cardboard and try how the packing fits
#
tray = pd.read_excel('Tray_New1.xlsx')  #2ml type
# tray = pd.read_excel('Tray_New2.xlsx') #5ml type
# tray = pd.read_excel('Tray_New3.xlsx')  #10ml type
# tray = pd.read_excel('Tray_New4.xlsx')  #20ml type

In [4]:
ABCDE = tuplelist(zip(tray["Tray"],tray["L"],tray["W"],tray["H"],tray["weight"]))

In [5]:
b = list(range(0,len(ABCDE)))

In [6]:
IJKLM = tuplelist(zip(data["MODEL"],data["cl"],data["cw"],data["ch"],data["Cap"]))

In [7]:
a = list(range(0,len(IJKLM)))

In [8]:
IJKLM = list(IJKLM)
ABCDE = list(ABCDE)

In [ ]:
packer = Packer()
for i in a:
    packer.add_bin(Bin(*IJKLM[i]))
# packer.add_bin(Bin('large-envelope', 15.0, 12.0, 0.75, 15))
# packer.add_bin(Bin('small-box', 8.625, 5.375, 1.625, 70.0))
# packer.add_bin(Bin('medium-box', 11.0, 8.5, 5.5, 70.0))
# packer.add_bin(Bin('medium-2-box', 13.625, 11.875, 3.375, 70.0))
# packer.add_bin(Bin('large-box', 12.0, 12.0, 5.5, 70.0))
# packer.add_bin(Bin('large-2-box', 23.6875, 11.75, 3.0, 70.0))
for j in b:
    packer.add_item(Item(*ABCDE[j]))
# packer.add_item(Item('50g [powder 2]', 3.9370, 1.9685, 1.9685, 2))
# packer.add_item(Item('50g [powder 3]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 4]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 5]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 6]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 7]', 5.1240, 1.1350, 1.5435, 3))
# packer.add_item(Item('250g [powder 8]', 7.8740, 3.9370, 1.9685, 4))
# packer.add_item(Item('250g [powder 9]', 7.8740, 3.9370, 1.9685, 5))
# packer.add_item(Item('250g [powder 10]', 7.8740, 3.9370, 1.9685, 6))
# packer.add_item(Item('250g [powder 11]', 7.8740, 3.9370, 1.9685, 7))
# packer.add_item(Item('250g [powder 12]', 7.8740, 3.9370, 1.9685, 8))
# packer.add_item(Item('250g [powder 13]', 7.8740, 3.9370, 1.9685, 9))

packer.pack()


::::::::::: S-16701(8.000x5.000x3.000, max_weight:30.000) vol(120.000) item_number(2) filling_ratio(0.254)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(0) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, 0]) rt(0) vol(15.249)

UNFITTED ITEMS:
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), 0, Decimal('1.531')]) rt(5) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3


::::::::::: S-16705(9.000x5.000x4.000, max_weight:30.000) vol(180.000) item_number(7) filling_ratio(0.593)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(1) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(3) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), 0, 0]) rt(3) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), Decimal('1.531'), 0]) rt(1) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), Decimal('1.531'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:
====>  8(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), Decimal('1.531'), Decimal('3.156')]) rt(5) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), Decimal('1.531'), Deci


::::::::::: S-15028(8.000x6.000x4.000, max_weight:30.000) vol(192.000) item_number(6) filling_ratio(0.477)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(3) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(3) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.218'), 0, 0]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), 0]) rt(3) vol(15.249)

UNFITTED ITEMS:
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), Decimal('3.156')]) rt(5) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), Decimal('3.156')]) rt(5) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal

::::::::::: S-19864(9.000x6.000x4.000, max_weight:30.000) vol(216.000) item_number(6) filling_ratio(0.424)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(3) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(3) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.218'), 0, 0]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), 0]) rt(3) vol(15.249)

UNFITTED ITEMS:
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), Decimal('3.156')]) rt(5) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal('3.156'), Decimal('3.156')]) rt(5) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.156'), Decimal(


::::::::::: S-19060(10.000x6.000x4.000, max_weight:30.000) vol(240.000) item_number(8) filling_ratio(0.508)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(1) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(3) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), 0, 0]) rt(3) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('7.749'), 0, 0]) rt(1) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), Decimal('1.531'), 0]) rt(3) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), Decimal('3.062'), 0]) rt(3) vol(15.249)

UNFITTED ITEMS:
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), Decimal('3.062'), Decimal('3.156')]) rt(5) vol(15.2

...


::::::::::: S-4424(14.000x12.000x12.000, max_weight:30.000) vol(2016.000) item_number(31) filling_ratio(0.234)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(0) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(1) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.156')]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.218'), 0, 0]) rt(0) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('6.312'), 0]) rt(1) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('6.312')]) rt(1) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('9.374'), 0, 0]) rt(0) vol(15.249)
====>  10(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('9.468'), 


::::::::::: S-23305(14.000x12.000x12.000, max_weight:30.000) vol(2016.000) item_number(31) filling_ratio(0.234)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(0) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(1) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.156')]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.218'), 0, 0]) rt(0) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('6.312'), 0]) rt(1) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('6.312')]) rt(1) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('9.374'), 0, 0]) rt(0) vol(15.249)
====>  10(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('9.468'),


::::::::::: S-16753(17.000x12.000x10.000, max_weight:30.000) vol(2040.000) item_number(31) filling_ratio(0.232)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(1) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(1) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.156')]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), 0, 0]) rt(1) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), 0, 0]) rt(3) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('6.312'), 0]) rt(1) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('6.312')]) rt(1) vol(15.249)
====>  10(3.156x3.156x1.531, weight: 0.190) pos([Decimal('9.280'), 0,


::::::::::: S-4810(32.000x8.000x8.000, max_weight:30.000) vol(2048.000) item_number(31) filling_ratio(0.231)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(1) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(1) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.156')]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), 0, 0]) rt(1) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), 0, 0]) rt(1) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('6.312'), 0]) rt(3) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('6.312')]) rt(0) vol(15.249)
====>  10(3.156x3.156x1.531, weight: 0.190) pos([Decimal('7.655'), 0, 0]


::::::::::: S-4304(32.000x10.000x6.500, max_weight:30.000) vol(2080.000) item_number(31) filling_ratio(0.227)
FITTED ITEMS:
====>  1(3.156x3.156x1.531, weight: 0.190) pos([0, 0, 0]) rt(1) vol(15.249)
====>  2(3.156x3.156x1.531, weight: 0.190) pos([Decimal('1.531'), 0, 0]) rt(1) vol(15.249)
====>  3(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.062'), 0, 0]) rt(1) vol(15.249)
====>  4(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('3.156'), 0]) rt(1) vol(15.249)
====>  5(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.156')]) rt(1) vol(15.249)
====>  6(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.593'), 0, 0]) rt(1) vol(15.249)
====>  7(3.156x3.156x1.531, weight: 0.190) pos([Decimal('6.124'), 0, 0]) rt(1) vol(15.249)
====>  8(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('6.312'), 0]) rt(1) vol(15.249)
====>  9(3.156x3.156x1.531, weight: 0.190) pos([Decimal('7.655'), 0, 0]) rt(1) vol(15.249)
====>  10(3.156x3.156x1.531, weight: 0.190) pos([Decimal('9.186'), 0, 0